<a href="https://colab.research.google.com/github/AshishGusain17/via_google_colab/blob/master/face_expressions_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# importing all the required packages for the code
%matplotlib inline
import pandas as pd
import os,shutil,math,scipy,cv2
import numpy as np
import matplotlib.pyplot as plt
import random as rn


from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix,roc_curve,auc

from PIL import Image
from PIL import Image as pil_image
from PIL import ImageDraw

from time import time
from glob import glob
from tqdm import tqdm
from skimage.io import imread
from IPython.display import SVG

from scipy import misc,ndimage
from scipy.ndimage.interpolation import zoom
# from scipy.ndimage import imread

import keras
from keras import backend as K
from keras.utils.np_utils import to_categorical
from keras import layers
from keras.preprocessing.image import save_img
from keras.utils.vis_utils import model_to_dot
from keras.applications.vgg16 import VGG16,preprocess_input
from keras.models import Sequential,Input,Model
from keras.layers import Dense,Flatten,Dropout,Concatenate,GlobalAveragePooling2D,Lambda,ZeroPadding2D
from keras.layers import SeparableConv2D,BatchNormalization,MaxPooling2D,Conv2D
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam,SGD
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard,CSVLogger,ReduceLROnPlateau,LearningRateScheduler

Using TensorFlow backend.


In [0]:
# code trained on google colab, so mounting the google drive for it
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# This function will be used later to plot the graphs for loss and accuracy of both training and test data-set
def show_final_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('acc')
    ax[1].plot(history.epoch, history.history["acc"], label="Train acc")
    ax[1].plot(history.epoch, history.history["val_acc"], label="Validation acc")
    ax[0].legend()
    ax[1].legend()

In [0]:
# Importing h5py package as my data is stored in the form of h5 file, so to extract the data.
# Now, the location of the file given in brackets is the location from the google drive.

import h5py
train = h5py.File('/content/drive/My Drive/face-dataset/img150.h5', "r")
test=   h5py.File('/content/drive/My Drive/face-dataset/img150_test.h5', "r")

X_train = train["X"][:]
Z_train = train["Z"][:]
name_train = train["name"][:]
X_test = test["X"][:]
Z_test = test["Z"][:]
name_test = test["name"][:]
# num = XX.shape[0] 
print(X_train.shape,Z_train.shape,X_test.shape,Z_test.shape,name_train.shape,name_test.shape)

In [0]:
import numpy as np
while 1:
  a=np.random.rand(100000,100000)

In [0]:
# Shape of each image is 150*150*3
imgsize=150

In [0]:
# Converting the indices into the categorical form
# Example: If 3 is the value of output, it gets converted to [0, 0, 1, 0, 0, 0, 0]
Z_train=to_categorical(Z_train,7)
Z_test=to_categorical(Z_test,7)
print(X_train.shape,Z_train.shape,X_test.shape,Z_test.shape)

In [0]:
# Here, using image data generator to get various variation of images from a single image.
augs_gen = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False, 
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        # rotation_range=20,  
        # zoom_range = 0.1, 
        # width_shift_range=0.2,  
        # height_shift_range=0.2, 
        # horizontal_flip=True,  
        vertical_flip=False) 
 
augs_gen.fit(X_train)

In [0]:
# Plotting, some of the face expressions randomly.
fig,ax=plt.subplots(5,5)
fig.set_size_inches(15,15)
for i in range(5):
    for j in range (5):
        l=rn.randint(0,len(X_train))
        ax[i,j].imshow(X_train[l])
        print(name_train[l][:])
        ax[i,j].set_title('Face-expressions: '+ str(name_train[l]))
        
plt.tight_layout()

In [0]:
# Import the VGG16 model which has some pre-trained weights.
# We require just the convolutional and pooling layers, not the end functional layers, so kept include_top equals to false.

base_model = keras.applications.mobilenet.MobileNet(include_top=False,
                  input_shape = (imgsize,imgsize,3),
                  weights = 'imagenet')

# Getting the summary of this model in the form of a chart
base_model.summary()

# Now, creating my own model and adding the layers of VGG16 to it 
model = Sequential()
for layer in base_model.layers:
  model.add(layer)

# Here, I am keeping my layers as trainable or not-trainable as per the accuracy I am getting.
for layer in base_model.layers[:-1]:
    layer.trainable = False
    print(layer,layer.trainable)

# Now, flattening all the layers into functional layers, earlier they were in rgb format.
model.add(Flatten())

# Now, I am adding the last functional dense layer which has the same number of neurons as total number of expressions we have, that is 7
model.add(Dense(7,activation='softmax'))
model.summary()

# Now, with the help of some graphs and pictures, visualising my model.
SVG(model_to_dot(model).create(prog='dot', format='svg'))
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [0]:
# Here, I have used some of the callbacks that are helpful during the training time.
# Tensorboard helps in visualising our various kinds of losses.
# Earlystop can be used if we are not getting improved result.
# csvlogger stores our epoch, training-accuracy, training-loss, validation-accuracy, validation-loss.

earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=50,
    verbose=1,
    mode='auto'
)
tensorboard = TensorBoard(
    log_dir = './logs',
    histogram_freq=0,
    batch_size=16,
    write_graph=True,
    write_grads=True,
    write_images=False,
)
checkpoint = ModelCheckpoint(
    './base.model',
    monitor='val_acc',
    verbose=1,
    save_best_only=True,
    mode='max',
    save_weights_only=False,
    period=1
)
csvlogger = CSVLogger(
    filename= "training_csv.log",
    separator = ",",
    append = False
)

reduce = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=3,
    verbose=1, 
    mode='auto'
)

callbacks = [earlystop,tensorboard,csvlogger,reduce]

In [0]:
# Here, I am using Adam as my adaptive learning rate optimization algorithm
opt = SGD(lr=1e-4,momentum=0.99)
opt1 = Adam(lr=1e-3)

# Now, compiling my model and providing it with the parameters like optimization technique, metrics and the loss
model.compile(
    loss='categorical_crossentropy',
    optimizer=opt1,
    metrics=['accuracy']
)

# Finally, training my model with the parameters given as below.
# Our batch-size is 32 and we are doing total 10 epochs and also shuffling is done after every batch training.
history = model.fit_generator(
    augs_gen.flow(X_train,Z_train,batch_size=32),
    validation_data  = (X_test,Z_test),
    # validation_steps = 100,
    # steps_per_epoch  = 100,
    epochs =40, 
    verbose = 1,
    callbacks=callbacks,
    shuffle=True
)

In [0]:
# This function was declared at the starting and is used for visualising the losses and accuracy.
show_final_history(history)

# Now, saving my model along with its weights.
model.save("model.h5")
print("Weights Saved")

In [0]:
# Here, I can test my entire test data set together 
a=model.evaluate(x_test,y_test)
print(a)
